<a href="https://colab.research.google.com/github/karynaur/Breaking-Bad/blob/main/Dhruva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq jmd_imagescraper

In [ ]:
from jmd_imagescraper.core import * 
from pathlib import Path

root = Path().cwd()/"images"


duckduckgo_search(root, "hank", "Dean Norris", max_results=20)
duckduckgo_search(root, "gustavo", "Giancarlo Esposito", max_results=20)

Duckduckgo search: Dean Norris


Duckduckgo search: Giancarlo Esposito


[PosixPath('/content/images/gustavo/021_db7f7abf.jpg'),
 PosixPath('/content/images/gustavo/022_09db72c7.jpg'),
 PosixPath('/content/images/gustavo/023_f955acad.jpg'),
 PosixPath('/content/images/gustavo/024_b2ee50f7.jpg'),
 PosixPath('/content/images/gustavo/025_27d7fc1f.jpg'),
 PosixPath('/content/images/gustavo/026_953ca849.jpg'),
 PosixPath('/content/images/gustavo/027_d89a6fbf.jpg'),
 PosixPath('/content/images/gustavo/028_a8733ace.jpg'),
 PosixPath('/content/images/gustavo/029_9b86eede.jpg'),
 PosixPath('/content/images/gustavo/030_46ed71db.jpg'),
 PosixPath('/content/images/gustavo/031_8bc6a4b4.jpg'),
 PosixPath('/content/images/gustavo/032_b2e0785b.jpg'),
 PosixPath('/content/images/gustavo/033_5ea2fbb0.jpg'),
 PosixPath('/content/images/gustavo/034_661febca.jpg'),
 PosixPath('/content/images/gustavo/035_a36708ac.jpg'),
 PosixPath('/content/images/gustavo/036_e5a2fa72.jpg'),
 PosixPath('/content/images/gustavo/037_c7bdc95b.jpg'),
 PosixPath('/content/images/gustavo/038_1f149530

In [ ]:
from jmd_imagescraper.imagecleaner import *

display_image_cleaner(root)

HTML(value='<h2>No images left to display in this folder.</h2>', layout=Layout(visibility='hidden'))

GridBox(children=(VBox(children=(Image(value=b'', layout="Layout(width='150px')"), Button(description='Delete'…

In [ ]:
import glob 
print(f"We have a total of {len(glob.glob('images/*/*'))} images")

We have a total of 78 images


In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline

In [ ]:
data = torchvision.datasets.ImageFolder(root = './images')

In [ ]:
data[46]

(<PIL.Image.Image image mode=RGB size=474x462 at 0x7FD349084290>, 1)

In [ ]:
len(data)

78

In [ ]:
 class BreakingBadDataset(Dataset):
    def __init__(self, root_dir):
        super().__init__()
        self.dataset = torchvision.datasets.ImageFolder(root = root_dir)
        self.transform = torchvision.transforms.Compose([
                    transforms.RandomResizedCrop((28,28)),
                    transforms.ToTensor()
                ])

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        item = self.dataset[idx]
        return (    
            self.transform(item[0]).view(28*28*3), 
            torch.tensor(item[1], dtype=torch.float)
        )

1. Training
2. Validation
3. Testing

In [ ]:
dataset = BreakingBadDataset('./images')

train_no = int(0.9 * len(dataset))
train_set, val_set = torch.utils.data.random_split(dataset, [train_no, len(dataset) - train_no])

In [ ]:
class DeviceDataLoader():
    def __init__(self, 
                 dataset, 
                 bs = 64, 
                 shuffle = True, 
                 device = 'cuda' if torch.cuda.is_available() else 'cpu'):
        self.dl = DataLoader(dataset, bs, shuffle, num_workers = 2)
        self.device = device

    def to_device(self, data, device):
        if isinstance(data, (list,tuple)):
            return [self.to_device(x, device) for x in data]
        return data.to(device, non_blocking=True)
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield self.to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
train_loader = DeviceDataLoader(train_set)
val_loader = DeviceDataLoader(val_set)

Now lets understand our data

1. Our dataset gives out image tensors of size (28\*28\*3)
2. Our output is a number (0,1)

<img src ="https://cdn.discordapp.com/attachments/856937388360990773/903622274156990474/unknown.png" height = 800px>

In [ ]:
class FeedForwardModel(nn.Module):
    def __init__(self, l = [28*28*3, 128, 1]):
        super().__init__()
        self.l1 = nn.Linear(l[0],l[1])
        self.l2 = nn.Linear(l[1],l[2]) 
        self.drop = nn.Dropout(0.09)
    
    def forward(self, x):
        x = self.l2(self.l1(x))
        return self.drop(x)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


def train_model(model, optim, train_dl, loss_fn):
    model.train()
    accuracy = 0
    total_loss = 0
    for  x, y in train_dl:
        batch = y.shape[0]
        output = model(x)
        acc = binary_acc(output, y.unsqueeze(1))

        loss = loss_fn(output, y.unsqueeze(1))
        loss.backward()
        optimizer.step()

        accuracy += acc
        total_loss += loss.item()
    return accuracy/len(train_dl), total_loss/len(train_dl)

def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

def val_loss(model, valid_dl, loss_fn):
    model.eval()
    total = len(valid_dl)
    sum_loss = 0
    acc = 0
    with torch.inference_mode():
        for x, y in valid_dl:
            out = model(x)
            loss = loss_fn(out, y.unsqueeze(1))
            sum_loss += loss.item()

            pred = torch.max(out, 1)[1]
            acc += binary_acc(pred, y.unsqueeze(1))
    print("valid loss %.3f and accuracy %.3f" % (sum_loss/total, acc/total))


In [ ]:
model = FeedForwardModel()
model.to(device)
optimizer=torch.optim.SGD(model.parameters(), lr=0.01, momentum=0)
loss_fn = nn.BCEWithLogitsLoss()


def train_loop(model, epochs, lr=0.01, wd=0.0):
    for i in range(epochs): 
        accuracy, loss = train_model(model, optimizer, train_loader, loss_fn)
        print("training loss: ", loss)
        val_loss(model, val_loader, loss_fn)

In [ ]:
train_loop(model, 10)

In [ ]:
torch.save(model.state_dict(), 'model.bin')